# Naive Bayes

**Naive Bayes** method is a classification method based on Bayes' theorem and independent assumptions of feature conditions. For a given training data set, first learn the joint probability distribution of the input and output based one the feature independent assumption; then based on this model, use the Bayes theorem to find the posterior probability $y$ for the given input $x$.

Supposed that input space $\chi \subseteq R^n$ is a collection of $n$ dimensional vectors, and the output space is a collection of class labels, $y = {c_c, c_2, ..., c_K}$. The input is feature vector, $x \in \chi$, and the output is class label, $y\in \texttt{Y}$. 

Training dataset:  $T = \{(x_1, y_1), (x_2, y_2), ..., (x_n, y_n)\}$

Prior probability distribution:  $P(Y=c_k), k=1, 2,..., k$

Conditional probability distribution:  $P(X=x|Y=c_k) = P(X^{(1)} = x^{(1)},..., X^{(n)} = x^{(n)}|Y=c_k), k = 1, 2, ...,K$

Therefore, we got the joint probability distribution: $P(X, Y)$